# Show Star and Planet Orientation

For explaining North vs South of map

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import starry
from copy import deepcopy
import astropy.units as u
import astropy.constants as const
from astropy.table import Table
from astropy.io import fits, ascii
from corner import corner
import os
import hotspot_fitter

starry.config.lazy = False
starry.config.quiet = True

In [ ]:
starry.__version__

## Set up Forward Model

Use Earth map as an example map

In [ ]:
max_ell = 3

up_bin = 10 ## up-bin for clarity
lc_precision = 145e-6 / np.sqrt(up_bin) ## measured from broadband F444W precision
cadence = 2.72486 * up_bin ## between exposures
npoints = int(np.round(0.2 * 24. * 3600.  / (cadence))) ## duration/cadence
extra_descrip = '_hd189733_f444w'

map1 = starry.Map(max_ell)

map1.load("earth", sigma=0.05)
y_input = deepcopy(map1.y[1:])
map1.show(projection="rect")

In [ ]:
modelDat = ascii.read('sim_data/hd189733b_spec_model_binned.csv')
modelDat[1]['depth']

In [ ]:
M_star = 0.812 ## Addison et al. 2019, solar masses
R_star = 0.765 ## Addison et al. 2019, solar radii
inc=85.69 ## Addison et al. 2019
#inc=90.0
rp = 0.1504 * R_star ## Addison et al. 2019, Solar radii
P_b = 2.218577 ## days
log_amp = np.log10(modelDat[1]['depth'])
t0 = 0.0

M_planet = (1.166 * u.Mjup).to(u.Msun).value ## Addison et al. 2019, Solar masses

prot_star = 1.0

x = np.linspace(0.5 * P_b - 0.1,0.5 * P_b + 0.1,npoints)
#x = np.linspace(0.,P_b,4096)


Save the brightness temperature for comparison with ThERESA results

Get the impact parameter

In [ ]:
norb = 2. * np.pi / (P_b * u.day)
a_orb = (const.G * (M_star * u.Msun) / norb**2)**(1./3.)
a_over_r_star = (a_orb/(R_star * u.Rsun)).si
b_impact = a_over_r_star * np.cos(inc * np.pi/180.)
b_impact

In [ ]:
a_orb.to(u.AU)

In [ ]:
arg = np.sqrt((1. + rp/R_star)**2 - b_impact**2)/ (a_over_r_star * np.sin(inc * np.pi/180.))
Tdur_14 = (P_b / np.pi) * np.arcsin(arg)
Thalf_14 = (0.5 * Tdur_14).value

arg2 = np.sqrt((1. - rp/R_star)**2 - b_impact**2)/ (a_over_r_star * np.sin(inc * np.pi/180.))
Tdur_23 = (P_b / np.pi) * np.arcsin(arg2)
Thalf_23 = (0.5 * Tdur_23).value
Thalf_14,Thalf_23

In [ ]:
A = starry.Primary(starry.Map(ydeg=0, udeg=2, amp=1.0), m=M_star, r=R_star,
                   prot=prot_star )
b = starry.kepler.Secondary(map1,
                            m=M_planet,r=rp,prot=P_b,porb=P_b,t0=t0,inc=inc)
b.map.amp = 10**log_amp

In [ ]:
b.theta0 = 180.0 + 0.0
sys = starry.System(A,b)

In [ ]:
#os.mkdir('plots/movies')

In [ ]:
sys.show(t=np.linspace(0.5 * P_b - 1.3 * Thalf_14,0.5 * P_b + 1.3 * Thalf_14,64),figsize=(8,8),
        file='plots/movies/eclipse_forward_model.mov')


In [ ]:
sys.show(t=np.linspace(0 * P_b - 1.3 * Thalf_14,0 * P_b + 1.3 * Thalf_14,64),figsize=(8,8),
        file='plots/movies/transit_forward_model.mov')

